In [11]:
import pandas as pd 
import numpy as np
from qiime2 import Artifact
import os

Import of the final sample dataset obtained in the EDA notebook: 

In [2]:
data = pd.read_csv('data_final_groups.csv',dtype = {"sample_name": "str"},index_col=0)
data

,sample_name,acid_reflux,acne_medication,acne_medication_otc,add_adhd,age_cat,age_corrected,age_years,alcohol_consumption,alcohol_frequency,...,vitamin_b_supplement_frequency,vitamin_d_supplement_frequency,vivid_dreams,weight_cat,weight_change,weight_kg,weight_units,whole_eggs,whole_grain_frequency,group
0,10317.000023139,No,No,No,No,60s,69.0,69.0,Yes,Rarely (a few times/month),...,Regularly (3-5 times/week),Daily,Never,NaN,Remained stable,63,kilograms,Regularly (3-5 times/week),Regularly (3-5 times/week),Treatment
1,10317.000027811,No,No,No,No,40s,46.0,46.0,Yes,Rarely (a few times/month),...,Rarely (a few times/month),Regularly (3-5 times/week),Rarely (a few times/month),NaN,Remained stable,81,kilograms,Occasionally (1-2 times/week),Rarely (less than once/week),Treatment
2,10317.000031513,Self-diagnosed,No,No,Not provided,30s,37.0,37.0,Yes,Regularly (3-5 times/week),...,Never,Rarely (a few times/month),Rarely (a few times/month),NaN,Remained stable,49,kilograms,Occasionally (1-2 times/week),Rarely (less than once/week),Treatment
3,10317.000033280,No,No,No,No,60s,60.0,60.0,Yes,Regularly (3-5 times/week),...,Never,Never,Never,NaN,Remained stable,68,kilograms,Occasionally (1-2 times/week),Regularly (3-5 times/week),Treatment
4,10317.000038261,No,No,No,No,40s,48.0,48.0,Yes,Regularly (3-5 times/week),...,Never,Never,Occasionally (1-2 times/week),NaN,Remained stable,70,kilograms,Rarely (less than once/week),Occasionally (1-2 times/week),Treatment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,10317.000065667,No,No,No,No,20s,22.0,22.0,Yes,Occasionally (1-2 times/week),...,Never,Never,Rarely (a few times/month),NaN,Remained stable,61,kilograms,Regularly (3-5 times/week),Never,Control
376,10317.000065673,No,No,Yes,No,20s,20.0,20.0,No,Never,...,Never,Never,Occasionally (1-2 times/week),NaN,Remained stable,79,kilograms,Never,Rarely (less than once/week),Control
377,10317.000065675,No,No,No,No,20s,21.0,21.0,No,Never,...,Never,Never,Daily,NaN,Remained stable,51,kilograms,Never,Regularly (3-5 times/week),Control
378,10317.000065681,No,No,Yes,No,20s,21.0,21.0,No,Never,...,Never,Never,Rarely (a few times/month),NaN,Remained stable,137,kilograms,Rarely (less than once/week),Rarely (less than once/week),Control


In [3]:
data.specialized_diet_exclude_refined_sugars.value_counts()

Yes    190
No     190
Name: specialized_diet_exclude_refined_sugars, dtype: int64

## Metadata file creation

In [4]:
metadata = data.sample_name.to_frame()

In [5]:
metadata = metadata.rename(columns={'sample_name': 'sample-id'})

In [6]:
# Aggiungiamo al manifest le informazioni riguardo l'età, il sesso e l'uso degli antibiotici
metadata['sex'] = data['sex']
metadata['age_cat'] = data['age_cat']
metadata['refined_sugars'] = data['specialized_diet_exclude_refined_sugars']
metadata

,sample-id,sex,age_cat,refined_sugars
0,10317.000023139,female,60s,Yes
1,10317.000027811,male,40s,Yes
2,10317.000031513,female,30s,Yes
3,10317.000033280,female,60s,Yes
4,10317.000038261,female,40s,Yes
...,...,...,...,...
375,10317.000065667,female,20s,No
376,10317.000065673,female,20s,No
377,10317.000065675,female,20s,No
378,10317.000065681,female,20s,No


In [7]:
metadata['sample-id'].to_csv("ids.txt",header=None,index=None)

In [8]:
metadata.refined_sugars.value_counts()

Yes    190
No     190
Name: refined_sugars, dtype: int64

In [9]:
metadata.to_csv('sample_metadata.tsv', sep='\t', index=False)

In [12]:
from qiime2 import Metadata
sample_metadata = Metadata.load(os.getcwd()+'/sample_metadata.tsv')

Import Fasta 99% otus containing Taxonomy

In [13]:
!qiime tools import --type 'FeatureData[Sequence]' --input-path gg_13_8_otus/rep_set/99_otus.fasta \
    --output-path rep_set.qza

Imported gg_13_8_otus/rep_set/99_otus.fasta as DNASequencesDirectoryFormat to rep_set.qza


In [15]:
otu_table_df = pd.read_csv("otu_table.tsv",sep="\t")

Aggiungere comando per tirare giù otutable

Aggiungere comando per trasformare .biom in .tsv

In [16]:
otu_table_df

,OTU ID,10317.000023139.25853,10317.000027811.25853,10317.000033280.31503,10317.000038261.33800,10317.000041075.48742,10317.000050189.26531,10317.000051222.53379,10317.000052017.26531,10317.000052017.31503,...,10317.000047222.26531,10317.000047387.26531,10317.000047463.25853,10317.000047540.64035,10317.000051556.56079,10317.000052323.26478,10317.000052472.50470,10317.000052511.31503,10317.000058850.25853,10317.000059011.26531
0,732765,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2233608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,676299,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4440335,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1076818,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5560,174911,29.0,0.0,0.0,1.0,52.0,0.0,5.0,0.0,0.0,...,2.0,1.0,2.0,0.0,0.0,0.0,30.0,0.0,1.0,1.0
5561,4430499,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5562,4466137,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5563,181689,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [117]:
!biom convert -i otu_table_filt.tsv -o otu_table_filt.biom --table-type="OTU table" --to-hdf5

In [55]:
!qiime feature-table filter-seqs --i-data rep_set.qza --m-metadata-file otu_filt.txt --o-filtered-data rep_set_filt.qza

Saved FeatureData[Sequence] to: rep_set_filt.qza


In [135]:
!biom convert -i otu_table_reorder.tsv -o otu_table_reorder.biom --table-type="OTU table" --to-hdf5

In [136]:
!qiime tools import \
  --input-path otu_table_reorder.biom\
  --type 'FeatureTable[Frequency]' \
  --input-format BIOMV210Format \
  --output-path feature_table.qza

Imported otu_table_reorder.biom as BIOMV210Format to feature_table.qza


In [18]:
from qiime2 import Artifact
otu_table = Artifact.load("feature_table.qza")
rep_set = Artifact.load("rep_set_filt.qza")

In [19]:
ot_df = otu_table.view(pd.DataFrame) 

In [112]:
ot_df_filt = ot_df.loc[ot_df.index.isin(meta_table_filt["sample-id"])]

In [115]:
ot_df_filt.to_csv("otu_table_filt.tsv",sep="\t")

In [ ]:
from qiime2.plugins import alignment, phylogeny
mafft_alignment = alignment.methods.mafft(rep_set)
masked_mafft_alignment = alignment.methods.mask(mafft_alignment.alignment)
unrooted_tree = phylogeny.methods.fasttree(masked_mafft_alignment.masked_alignment)
rooted_tree = phylogeny.methods.midpoint_root(unrooted_tree.tree)


In [5]:
rooted_tree = Artifact.load("rooted_tree.qza")

In [7]:
from qiime2 import Metadata
metadata = Metadata.load("sample_metadata.tsv")

In [73]:
meta_table = pd.read_csv("sample_metadata.tsv",sep="\t",dtype = {"sample-id": "str"})

In [74]:
meta_table = meta_table.sort_values(by="sample-id").reset_index(drop=True)

In [47]:
otu_index = otu_table.view(pd.DataFrame).index.sort_values()

In [77]:
non_match = []
for i in range(meta_table.shape[0]):
    result = [k for k in otu_index if k.startswith(meta_table["sample-id"][i])]
    if len(result) == 0:
        non_match.append(meta_table["sample-id"][i] )
    else: 
        meta_table["sample-id"][i] = meta_table["sample-id"][i] + result[0][-6:]

In [90]:
meta_table_filt = meta_table.loc[~meta_table["sample-id"].isin(non_match)] 
    

In [92]:
meta_table_filt.to_csv("sample_metadata.tsv",sep="\t",index=None)

In [132]:
otu_df = otu_table.view(pd.DataFrame)

,10317.000001166.25853,10317.000003100.69016,10317.000004936.25853,10317.000010862.25853,10317.000012018.56079,10317.000013727.69016,10317.000014885.41931,10317.000017272.30606,10317.000020655.26478,10317.000022184.26478,...,10317.000107324.71215,10317.000107446.76952,10317.000107636.74729,10317.000107686.76952,10317.000108028.74729,10317.000108434.71215,10317.000108496.74729,10317.000108903.74729,10317.000108906.74729,10317.000109791.74729
732765,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2233608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
676299,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4440335,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1076818,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174911,25.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,7.0,2.0,...,1.0,1.0,0.0,10.0,15.0,0.0,0.0,2.0,0.0,1.0
4430499,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4466137,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
181689,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [133]:
otu_reorder = otu_df.reindex(sorted(otu_df.columns), axis=1)

In [134]:
otu_reorder.to_csv("otu_table_reorder.tsv",sep="\t")

In [3]:
from qiime2.plugins import feature_table

In [4]:
output_viz = feature_table.visualizers.summarize(otu_table)
output_viz.visualization.save("viz.qzv")

'viz.qzv'

In [67]:
from qiime2.plugins import diversity
core_metrics = diversity.pipelines.core_metrics_phylogenetic(table = otu_table,
                                                             phylogeny = rooted_tree,
                                                             sampling_depth = 8200,
                                                             metadata = metadata)

/opt/anaconda3/envs/qiime2-2023.2/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:1776: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/opt/anaconda3/envs/qiime2-2023.2/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.2078278690323221 and the largest is 13.850997935921855.
  warn(


Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command:

faithpd -i /tmp/qiime2/gc882638/data/67a7aee2-ebd9-4917-9225-50af83e90258/data/feature-table.biom -t /tmp/qiime2/gc882638/data/16730575-37b3-4cb2-905f-d568d0d7eb7c/data/tree.nwk -o /tmp/q2-AlphaDiversityFormat-_pirtrnl

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command:

ssu -i /tmp/qiime2/gc882638/data/67a7aee2-ebd9-4917-9225-50af83e90258/data/feature-table.biom -t /tmp/qiime2/gc882638/data/16730575-37b3-4cb2-905f-d568d0d7eb7c/data/tree.nwk -m unweighted -o /tmp/q2-LSMatFormat-ar_uot7d

Running external command line application. This may print messages to stdout and/or stderr.
T

/opt/anaconda3/envs/qiime2-2023.2/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.40203254500677355 and the largest is 13.996728272991078.
  warn(


In [68]:
import pandas as pd

In [69]:
core_metrics.rarefied_table.view(pd.DataFrame)

,732765,4440335,1076818,288690,157894,513021,4362724,22928,308419,298346,...,369668,184729,192936,4375646,291445,195882,174911,4430499,181689,4426367
10317.000001166.25853,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0
10317.000003100.69016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000010862.25853,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000012018.56079,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000013727.69016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10317.000108434.71215,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
10317.000108496.74729,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000108903.74729,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000108906.74729,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [70]:
core_metrics_rarefied_table_df = core_metrics.rarefied_table.view(pd.DataFrame).reset_index().melt(id_vars=['index'],
                                                          var_name="feature", 
                                                          value_name="Value")
core_metrics_rarefied_table_df.to_csv('core_metrics_rarefied_table_df.csv')
core_metrics_rarefied_table_df

,index,feature,Value
0,10317.000001166.25853,732765,0.0
1,10317.000003100.69016,732765,0.0
2,10317.000010862.25853,732765,0.0
3,10317.000012018.56079,732765,0.0
4,10317.000013727.69016,732765,0.0
...,...,...,...
814193,10317.000108434.71215,4426367,0.0
814194,10317.000108496.74729,4426367,0.0
814195,10317.000108903.74729,4426367,0.0
814196,10317.000108906.74729,4426367,0.0


In [143]:
core_metrics_rarefied_table_df

0            0.0
457378       0.0
457379       0.0
457380       0.0
457381       0.0
           ...  
307679    2992.0
201017    2994.0
307514    3084.0
307512    3167.0
307523    3205.0
Name: Value, Length: 696276, dtype: float64

In [ ]:
core_metrics.o

In [71]:
observed_otus_vector_group_significance = diversity.actions.alpha_group_significance(core_metrics.observed_otus_vector,
                                                                                     metadata)
observed_otus_vector_group_significance.visualization

AttributeError: 'Results' object has no attribute 'observed_otus_vector'

In [73]:
shannon_vector_group_significance = diversity.actions.alpha_group_significance(core_metrics.shannon_vector,
                                                                         metadata)
shannon_vector_group_significance.visualization.save("shannon.qzv")

'shannon.qzv'

In [76]:
faith_pd_group_significance = diversity.actions.alpha_group_significance(core_metrics.faith_pd_vector,
                                                                         metadata)
faith_pd_group_significance.visualization.save("faith_pd.qzv")

'faith_pd.qzv'

In [78]:
evenness_group_significance = diversity.actions.alpha_group_significance(core_metrics.evenness_vector,
                                                                         metadata)
evenness_group_significance.visualization.save("evenness_group.qzv")

'evenness_group.qzv'

In [80]:
bray_curtis_distance_significance = diversity.actions.beta_group_significance(core_metrics.bray_curtis_distance_matrix,
                                                                           metadata.get_column('refined_sugars'))
bray_curtis_distance_significance.visualization.save("bray_curtis.qzv")

'bray_curtis.qzv'

In [81]:
jaccard_distance_significance = diversity.actions.beta_group_significance(core_metrics.jaccard_distance_matrix,
                                                                           metadata.get_column('refined_sugars'))
jaccard_distance_significance.visualization.save("jaccard_beta.qzv")

'jaccard_beta.qzv'

In [1]:
unweighted_unifrac_significance = diversity.actions.beta_group_significance(core_metrics.unweighted_unifrac_distance_matrix,
                                                                           sample_metadata.get_column('antibiotic_history_ord'))
unweighted_unifrac_significance.visualization.save("unweighted_unifrac.qzv")

NameError: name 'diversity' is not defined

In [ ]:
weighted_unifrac_significance = diversity.actions.beta_group_significance(core_metrics.weighted_unifrac_distance_matrix,
                                                                           sample_metadata.get_column('antibiotic_history_ord'))
weighted_unifrac_significance.visualization.save("weighted_unifrac.qzv")

In [ ]:
emperor_plot = emperor.visualizers.plot(core_metrics.unweighted_unifrac_pcoa_results,
                                        sample_metadata)
                                        #custom_axes=['DaysSinceExperimentStart'])
emperor_plot.visualization

In [ ]:
gut_deblur = feature_table.methods.filter_samples(deblur_sequences.table,
                                                  metadata = sample_metadata)

In [ ]:
deblur_composition = composition.actions.add_pseudocount(deblur_sequences__table)

ancom_deblur = composition.actions.ancom(table = deblur_composition.composition_table,
                                             metadata = sample_metadata.get_column('antibiotic_history'))

In [ ]:
ancom_deblur.visualization